# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
# Importing dependencies
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute

from azureml.core.run import Run
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Environment, ScriptRunConfig


import os
import joblib
import logging
import argparse

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'ClassifyHeartFailure-HyperDrive'

experiment=Experiment(ws, experiment_name)

# Getting data in tabular form from the path
path = "https://github.com/ShashiChilukuri/Data2Deployment-AzureML/blob/bc2160defa96caf4a11b509e1d7ce59a47bab792/heart_failure_clinical_records_dataset.csv"
data = TabularDatasetFactory.from_delimited_files(path)

In [ ]:
# Check if. cluster exists, if not create one
cluster_name = "Compute-Standard"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',min_nodes=1, max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

# get status of the cluster
print(cpu_cluster.get_status().serialize())

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
parameter_space = {"--n_estimators": choice(10, 20, 40), "--min_samples_split": choice(2,4,6)}
param_sampling = RandomParameterSampling(parameter_space = parameter_space)

#TODO: Create your estimator and hyperdrive config
# estimator = <your estimator here>

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = ".",
                      script='train.py',
                      compute_target=cluster_name,
                      environment = sklearn_env)


hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                         hyperparameter_sampling=param_sampling,
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal= PrimaryMetricGoal("MAXIMIZE"),
                                         max_total_runs=20,
                                         max_concurrent_runs=4,
                                         policy=early_termination_policy)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_hd_run = hyperdrive_run.get_best_run_by_primary_metric()
best_hd_run_metrics = best_hyperdrive_run.get_metrics()

print('Best Run Id: ', best_hyperdrive_run.id)
print('\n Best Run Metrics: ', best_hd_run_metrics)

In [ ]:
#TODO: Save the best model
os.makedirs("./outputs", exist_ok=True)
joblib.dump(value=best_hd_run.id,filename='outputs/best_hyperdrive_run_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_hd_run.register_model(model_name='heart_failure_hyperdrive', 
                                   model_path='outputs/', 
                                   properties={'Accuracy': best_hd_run_metrics['Accuracy'],
                                               'N Estimators': best_hd_run_metrics['N.O trees in the forest:'],
                                               'Min Samples Split': best_hd_run_metrics['Min samples to split:']})

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

